# Setup and Obtain the Data 
## Import Libraries and Dependencies

In [2]:
%pip install labelme tensorflow tensorflow-gpu opencv-python matplotlib albumentations pillow

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/44/jj50yyvd6fb9m109s2slr96w0000gn/T/pip-install-v81whnsx/tensorflow-gpu_0b055becd2534ce4b4fc427371bb49a5/setup.py", line 37, in <module>
          raise Exception(TF_REMOVAL_WARNING)
      Exception:
      
      The "tensorflow-gpu" package has been removed!
      
      Please install "tensorflow" instead.
      
      Other than the name, the two packages have been identical
      since TensorFlow 2.1, or roughly since Sep 2019. For more
      information, see: pypi.org/project/tensorflow-gpu
      
      
      [end of output]
  
  note: This error originates from a subp

## Collect Images Using OpenCV

In [3]:
import os  # for accessing the file system
import time  # for measuring the inference time per image
import uuid  # for generating unique IDs for our images
import cv2  # for computer vision tasks

In [4]:
IMAGES_PATH = os.path.join('Tensorflow', 'images')
number_of_images = 50

In [10]:
# Collecting Images and name them with our unique IDs
capture = cv2.VideoCapture(0)
for i in range(number_of_images):
    ret, frame = capture.read()
    # Generate a unique ID for our image
    img_id = uuid.uuid1()
    img_name = os.path.join(IMAGES_PATH, f'image_{img_id}.jpg')
    # Save the image
    cv2.imwrite(img_name, frame)
    cv2.imshow('frame', frame)
    time.sleep(0.5)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    print(f'Image {i} saved')
# Release the camera resource and desstroy the window opened by OpenCV
capture.release()
cv2.destroyAllWindows()

Image 0 saved
Image 1 saved
Image 2 saved
Image 3 saved
Image 4 saved
Image 5 saved
Image 6 saved
Image 7 saved
Image 8 saved
Image 9 saved
Image 10 saved
Image 11 saved
Image 12 saved
Image 13 saved
Image 14 saved
Image 15 saved
Image 16 saved
Image 17 saved
Image 18 saved
Image 19 saved
Image 20 saved
Image 21 saved
Image 22 saved
Image 23 saved
Image 24 saved
Image 25 saved
Image 26 saved
Image 27 saved
Image 28 saved
Image 29 saved
Image 30 saved
Image 31 saved
Image 32 saved
Image 33 saved
Image 34 saved
Image 35 saved
Image 36 saved
Image 37 saved
Image 38 saved
Image 39 saved
Image 40 saved
Image 41 saved
Image 42 saved
Image 43 saved
Image 44 saved
Image 45 saved
Image 46 saved
Image 47 saved
Image 48 saved
Image 49 saved


## Annotating Images with Bounding Boxes in LabelMe

In [ ]:
!labelme